# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with To import the file from Resources: mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

In [3]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [4]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [5]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'gardenDB', 'local', 'met', 'mykong', 'plants', 'travel_db', 'uk_food']


In [6]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [7]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [8]:
# review a document in the establishments collection
establishments = db.establishments
pprint(db.establishments.find_one())

{'AddressLine1': 'St Marys Church Of England Primary School',
 'AddressLine2': 'Warren Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'St Marys COE (aided) Primary School',
 'BusinessType': 'School/college/university',
 'BusinessTypeID': 7845,
 'ChangesByServerID': 0,
 'Distance': 4591.871474174837,
 'FHRSID': 289353,
 'LocalAuthorityBusinessID': 'PI/000002468',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6QH',
 'RatingDate': '2016-09-13T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('671c79062e746b1c20de20a0'),
 'geocode': {'latitude': '51.085797', 'longitude': '1.194762'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/289353',
         

In [9]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [10]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "1",
    "address": "123 Yummy Way, Greenwich",
    "rating": 5,
}

In [11]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

# Now after insert_one new_restaurant has additional property, _id
print(new_restaurant)

{'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': '1', 'address': '123 Yummy Way, Greenwich', 'rating': 5, '_id': ObjectId('671efd28f5917d676e7cca07')}


In [12]:
# Check that the new restaurant was inserted

establishments.find_one({"_id": new_restaurant["_id"]})

{'_id': ObjectId('671efd28f5917d676e7cca07'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '1',
 'address': '123 Yummy Way, Greenwich',
 'rating': 5}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [13]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
fields = {"BusinessTypeID": 1, "BusinessType": 1}
results = establishments.find(query, fields).limit(5)
for result in results:
    print(result)
    


{'_id': ObjectId('671c79062e746b1c20de20ab'), 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'_id': ObjectId('671c79062e746b1c20de20ae'), 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'_id': ObjectId('671c79062e746b1c20de20af'), 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'_id': ObjectId('671c79062e746b1c20de20b0'), 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}
{'_id': ObjectId('671c79062e746b1c20de20c1'), 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [14]:
# Update the new restaurant with the correct BusinessTypeID
query = {"BusinessName": "Penang Flavours"}
data = {"$set": {"BusinessTypeID": 1}}
establishments.update_one(query, data)


UpdateResult({'n': 1, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [19]:
# Confirm that the new restaurant was updated
updated_restaurant = establishments.find_one(query)
establishments.find_one({"_id": updated_restaurant["_id"]})
print(updated_restaurant)

{'_id': ObjectId('671c7cf29ea572ff22a5fa90'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'address': '123 Yummy Way, Greenwich', 'rating': 5}


In [20]:
#Check total documents in the collection
total_count = establishments.count_documents({})
print("Total documents in collection:", total_count)

Total documents in collection: 38790


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [22]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}
count = establishments.count_documents(query)
print(f"Number of documents with LocalAuthorityName Dover is: {count}")

Number of documents with LocalAuthorityName Dover is: 0


In [23]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {"LocalAuthorityName": "Dover"}
establishments.delete_many(query)

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [24]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName": "Dover"}
#establishments.find_one(query)
count = establishments.count_documents(query)
print(count)


results = establishments.find(query)
for result in results:
    print(result)

0


In [25]:
# Check that other documents remain with 'find_one'
query = {"LocalAuthorityName": {"$exists": True}}
documents = establishments.find_one(query)
print(documents)

{'_id': ObjectId('671c79062e746b1c20de20a0'), 'FHRSID': 289353, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000002468', 'BusinessName': 'St Marys COE (aided) Primary School', 'BusinessType': 'School/college/university', 'BusinessTypeID': 7845, 'AddressLine1': 'St Marys Church Of England Primary School', 'AddressLine2': 'Warren Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6QH', 'Phone': '', 'RatingValue': 5, 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2016-09-13T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.194762', 'latitude': '51.085797'}, 'RightToReply': '', 'Distance': 4591.871474174837, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractD

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [30]:
# Change the data type from String to Decimal for longitude and latitude

establishments.update_many(
    {"LocalAuthorityName": {"$exists": True}},
    [
        {
            '$set': {
                "geocode.latitude": {'$toDouble': "$geocode.latitude"},
                "geocode.longitude": {'$toDouble': "$geocode.longitude"}
            }
        }
    ]
)


# Verify the results
results = establishments.find({"LocalAuthorityName": {"$exists": True}}, {"geocode.latitude": 1, "geocode.longitude": 1}).limit(5)

# Print each document
for doc in results:
    print(doc)

{'_id': ObjectId('671c79062e746b1c20de20a0'), 'geocode': {'longitude': 1.194762, 'latitude': 51.085797}}
{'_id': ObjectId('671c79062e746b1c20de20a1'), 'geocode': {'longitude': 1.188537, 'latitude': 51.08084}}
{'_id': ObjectId('671c79062e746b1c20de20a3'), 'geocode': {'longitude': 1.188537, 'latitude': 51.08084}}
{'_id': ObjectId('671c79062e746b1c20de20a4'), 'geocode': {'longitude': 1.18590330311705, 'latitude': 51.0783519967076}}
{'_id': ObjectId('671c79062e746b1c20de20a5'), 'geocode': {'longitude': 1.188537, 'latitude': 51.08084}}


In [36]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, {'$set': { "RatingValue": None }})

#Verify the results:
results = establishments.find({"RatingValue": None}, {"RatingValue": 1, "_id": 0}).limit(5)
# Print each document
for doc in results:
    print(doc)

{'RatingValue': None}
{'RatingValue': None}
{'RatingValue': None}
{'RatingValue': None}
{'RatingValue': None}


In [37]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {"RatingValue": {"$type": "string", "$regex": "^[1-5]$"}},  # Match strings that are numeric 1-5
    [
        {'$set': {
            "RatingValue": {'$toInt': "$RatingValue"}
        }}
    ]
)

# Verify the results
results = establishments.find({"RatingValue": {"$exists": True}},{"RatingValue": 1, "_id": 0}).limit(5)

# Print each document
for doc in results:
    print(doc)

{'RatingValue': 5}
{'RatingValue': 5}
{'RatingValue': 5}
{'RatingValue': 5}
{'RatingValue': 5}


In [40]:
# Check that the coordinates and rating value are now numbers
result = establishments.find_one({
    "geocode.latitude": {"$type": "double"},
    "geocode.longitude": {"$type": "double"},
    "RatingValue": {"$type": "int"}
})

print(f"Latitude: {result['geocode']['latitude']} (Type: {type(result['geocode']['latitude'])})")
print(f"Longitude: {result['geocode']['longitude']} (Type: {type(result['geocode']['longitude'])})")
print(f"RatingValue: {result['RatingValue']} (Type: {type(result['RatingValue'])})")

Latitude: 51.085797 (Type: <class 'float'>)
Longitude: 1.194762 (Type: <class 'float'>)
RatingValue: 5 (Type: <class 'int'>)
